<a href="https://colab.research.google.com/github/xyluo25/DLSim-MRM/blob/main/DLSim.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Step 1 Install DLSim onto virtual machine (Linux)

In [1]:
!pip install DLSim

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 405.4/405.4 kB 4.7 MB/s eta 0:00:00


# Step 2 Clone data from github for test


In [2]:
! git clone https://github.com/xyluo25/DLSim-MRM.git

Cloning into 'DLSim-MRM'...
remote: Enumerating objects: 1392, done.
remote: Counting objects: 100% (316/316), done.
remote: Compressing objects: 100% (231/231), done.
remote: Total 1392 (delta 139), reused 231 (delta 75), pack-reused 1076
Receiving objects: 100% (1392/1392), 329.08 MiB | 21.47 MiB/s, done.
Resolving deltas: 100% (640/640), done.


# Step 3 Test Datasets

## Test first dataset

In [3]:
# show current working directory
! pwd

/content


In [4]:
# navigate the folder to our test data folder
%cd DLSim-MRM/datasets/ASU

/content/DLSim-MRM/datasets/ASU


In [5]:
! pwd

/content/DLSim-MRM/datasets/ASU


In [6]:
# import DLSim
from DLSim import DLSim
DL = DLSim()

DLSim version: 0.2.11


In [7]:
# check the currently working environment
DL.check_working_directory("/content/DLSim-MRM/datasets/ASU")


Current working directory: /content/DLSim-MRM/datasets/ASU.             Please note that all the input files ['settings.csv', 'node.csv', 'link.csv', 'demand.csv'] should be in this directory.             You can change the working directory by calling the function:             check_working_directory(working_dir = 'your working directory')


In [8]:
# check inputs 
DL.check_DLSim_input_files()

input dir: /content/DLSim-MRM/datasets/ASU 
input file type csv
All the required files ['settings.csv', 'node.csv', 'link.csv', 'demand.csv'] in the working directory.


In [9]:
DL.DLSim_settings

Loading settings from /content/DLSim-MRM/datasets/ASU/settings.csv.             you can prepare or change the settings.csv file in the working directory.


,[assignment],Unnamed: 1,assignment_mode,number_of_iterations,column_updating_iterations,signal_updating_iterations,signal_updating_output,remarks
0,NaN,NaN,ue,40,40,-1,0,"assignment_mode can be ue, dta or odme"
1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,[agent_type],agent_type,name,NaN,VOT,flow_type,PCE,NaN
3,NaN,p,passenger,NaN,10,0,1,NaN
4,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5,[link_type],link_type,link_type_name,NaN,agent_type_blocklist,type_code,traffic_flow_code,NaN
6,NaN,1,Highway/Expressway,NaN,NaN,f,0,NaN
7,NaN,2,Major arterial,NaN,NaN,a,0,NaN
8,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
9,[demand_period],demand_period_id,demand_period,NaN,time_period,NaN,NaN,NaN


In [10]:
DL.perform_kernel_network_assignment_simulation()

INFO Begin to run function: perform_kernel_network_assignment_simulation …
The default assignment mode is 2 (UE + DTA)                and the default column generation number is 10                and the default column update number is 10.
assignment_mode:  2
column_gen_num:  10
column_update_num:  10

DTALite run starts

DTALite run completes

check link_performance.csv in /content/DLSim-MRM/datasets/ASU for link performance
check agent.csv in /content/DLSim-MRM/datasets/ASU for unique agent paths

INFO Finished running function: perform_kernel_network_assignment_simulation, total: 3s

